In [20]:
import os
import getpass
from langchain.agents import create_openai_functions_agent
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage

In [2]:
llm = ChatOpenAI(model="gpt-3.5-turbo-1106", streaming=True)

### Defining Agent State

In [ ]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    sender: str

In [31]:
USER_SYSTEM_PROMPT = """You are a translator. Translate the text provided by the user into {user_language}. Output only the 
    translated text. If the text is already in {user_language}, return the user's text as it is.
"""


class UserAgent(object):

    def __init__(self, llm, prompt):
        self.llm = llm
        self.prompt = prompt
        self.chain = prompt | llm

        self.messages = []

    def invoke(self, user_text):

        output = self.chain.invoke({'user_text':['What is your name?']})

        return output
            
        
def create_user_agent(llm, user_language):

    system_prompt = USER_SYSTEM_PROMPT.format(user_language = user_language)
    
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system", system_prompt,
            ),
            MessagesPlaceholder(variable_name = 'user_text'),
        ]
    )

    agent = UserAgent(llm, prompt)

    return agent
    
    
    

In [32]:
from langgraph.graph import StateGraph, END

In [33]:
french_agent = create_user_agent(llm, "French")
spanish_agent = create_user_agent(llm, "Spanish")

In [34]:
french_agent.invoke('What is your name?')

AIMessage(content='Quel est votre nom ?', response_metadata={'finish_reason': 'stop'}, id='run-ca6ac45d-d29e-47c5-b3df-f84e9855ad6a-0')

In [18]:
prompt

ChatPromptTemplate(input_variables=[], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="You are a translator. Translate the text provided by the user into French. Output only the \n    translated text. If the text is already in French, return the user's text as it is.\n"))])

In [22]:
system_prompt = USER_SYSTEM_PROMPT.format(user_language = user_language)
    
prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system", system_prompt,
            ),
            MessagesPlaceholder(variable_name = 'user_text'),
        ]
    )

In [24]:
prompt.invoke({'user_text':['What is your name?']})

ChatPromptValue(messages=[SystemMessage(content="You are a translator. Translate the text provided by the user into French. Output only the \n    translated text. If the text is already in French, return the user's text as it is.\n"), HumanMessage(content='What is your name?')])

In [25]:
chain = prompt | llm

In [26]:
chain.invoke({'user_text':['What is your name?']})

AIMessage(content='Quel est ton nom ?', response_metadata={'finish_reason': 'stop'}, id='run-6cf9fd5d-39f4-49a6-a3df-2b78f74fa40d-0')

### Defining Agent Nodes